In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error

import random
import numpy as np
import DataPreprocessing
import SmartPolynomials

Using TensorFlow backend.


In [2]:
combs = ['1,3','1,4','2,3','2,4','1,2,3','1,2,4','1,3,4','2,3,4']

In [3]:
me_train, mom_train = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_1000000', 'LO/MG_uuuxux/mom_4jet_1000000', combs, frac=1)
me_test, mom_test = DataPreprocessing.npy('LO/MG_uuuxux/me_4jet_100000', 'LO/MG_uuuxux/mom_4jet_100000', combs, frac=0.1)
#me_train, mom_train = DataPreprocessing.npy('LO/MG_uuxgg/JOE_me_3000000.npy', 'LO/MG_uuxgg/JOE_mom_3000000.npy', combs, frac=1)
#me_test, mom_test = DataPreprocessing.npy('LO/MG_uuxgg/JOE_me_500000.npy', 'LO/MG_uuxgg/JOE_mom_500000.npy', combs, frac=0.1)
me_train=me_train/10.0**30
me_test=me_test/10.0**30

In [4]:
# Random Model from Tensorflow Website
def build_model(components=200, momentum=0.99, learning_rate=0.000002):
  model = keras.Sequential([
    keras.layers.Dense(components, activation='tanh', input_shape=[12,]), #could try swish??  
    
    keras.layers.Dense(components, activation='tanh'),
    keras.layers.Dense(components, activation='tanh'),
      
    keras.layers.Dense(1, activation='linear') 
  ])

  sgd=tf.keras.optimizers.SGD(0.000002, momentum=momentum, nesterov=True)
  model.compile(loss='mse', 
                optimizer=sgd,
                metrics=['mse'])
  return model

In [5]:
def run_model(params):
    step=100000

    model = build_model(*params)
    scaler = StandardScaler()

    training_error = []
    test_error = []
    perc_error = []

    for n in range(0,len(mom_train),step):
        poly_train = np.array([np.ndarray.flatten(np.array(element)) for element in mom_train[n:n+step,1:]])
        ##Rescale
        scaler.partial_fit(poly_train) #Rescale on training set
        poly_train = scaler.transform(poly_train)

        model.fit(poly_train,me_train[n:n+step], batch_size=64, epochs=100, shuffle=True,verbose=0)

        if(n%100000==0): 
            print(n)

            ##Testing Error
            lin_mse = 0
            lin_perc = 0

            for m in range(0,len(mom_test),step):
                print(m)

                poly_test = np.array([np.ndarray.flatten(np.array(element)) for element in mom_test[m:m+step,1:]])
                poly_test = scaler.transform(poly_test)

                me_predict = model.predict(poly_test, verbose=0) #Prediction on test set

                lin_mse += np.sum(mean_squared_error(me_test[m:m+step], me_predict))
                lin_perc += np.sum(100*np.divide(np.abs(me_test[m:m+step] - me_predict), me_test[m:m+step]))

            test_error.append(np.sqrt(lin_mse*step/len(me_test)))

            u=model.evaluate(poly_train, me_train[n:n+step])[0]
            training_error.append(np.sqrt(u))
            perc_error.append(lin_perc/len(me_test))

            print(training_error)
            print(test_error)
            print(perc_error)
    return(training_error, test_error, perc_error)

In [6]:
# specify parameters and distributions to sample from
components = [200,250,300]
momentum = [0.95,0.99,0.995,0.999]
learning_rate = [0.000001,0.000002,0.000003,0.000005]

final_error = []
# run randomized search
for i in range(20):
    params = [random.choice(components), random.choice(momentum), random.choice(learning_rate)]
    print(params)
    temp=run_model(params)
    final_error.append(temp)

[200, 0.999, 2e-06]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
0
0
100000/100000 [==============================]100000/100000 [==============================] - 3s 28us/step

[37.224402252116064]
[116.54940471416132198]
[7201972.138920282232]
100000
0
100000/100000 [==============================]100000/100000 [==============================] - 3s 29us/step

[37.224402252116064, 37.08452125574197]
[116.54940471416132198, 116.95035254328737574]
[7201972.138920282232, 6166764.6022812340484]
200000
0
100000/100000 [==============================]100000/100000 [==============================] - 3s 28us/step

[37.224402252116064, 37.08452125574197, 36.88253188187571]
[116.54940471416132198, 116.95035254328737574, 116.340469471800586676]
[7201972.138920282232, 6166764.6022812340484, 6594432.8820656565995]
300000
0
100000/100000 [==============================]100000/100000 [==============================] - 3s 28us/step

[37.224402252116064, 37.08452125574197, 

KeyboardInterrupt: 